In [ ]:
import os
import tifffile as tiff
import matplotlib.pyplot as plt
import math

# === Set your directory here ===
directory = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um"

def display_tiff_images(directory):
    if not os.path.isdir(directory):
        print("Invalid directory.")
        return

    tiff_files = [f for f in os.listdir(directory) if f.lower().endswith(('.tif', '.tiff'))]

    if not tiff_files:
        print("No TIFF images found.")
        return

    images_to_plot = []

    for filename in tiff_files:
        filepath = os.path.join(directory, filename)
        try:
            image = tiff.imread(filepath)

            if image.ndim == 2:
                images_to_plot.append((image, f"{filename} (slice 1)"))
            elif image.ndim == 3:
                for i in range(image.shape[0]):
                    images_to_plot.append((image[i], f"{filename} (slice {i + 1})"))
            else:
                print(f"Skipping {filename}: unsupported shape {image.shape}")
        except Exception as e:
            print(f"Failed to load {filename}: {e}")

    # === Plot in rows of 5 ===
    cols = 5
    rows = math.ceil(len(images_to_plot) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()

    for i, (img, title) in enumerate(images_to_plot):
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(title, fontsize=8)
        axes[i].axis('off')

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

# === Run the function ===
display_tiff_images(directory)


In [ ]:
import cv2
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# === Load original float images ===
fe_img = tiff.imread("/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Fe_merged.tiff").astype(np.float32)
ca_img = tiff.imread("/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Ca_merged.tiff").astype(np.float32)
s_img  = tiff.imread("/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_S_merged.tiff").astype(np.float32)

# === Normalize to uint8 ===
def normalize_img(img):
    norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    return norm_img.astype(np.uint8)

fe_norm = normalize_img(fe_img)
ca_norm = normalize_img(ca_img)
s_norm  = normalize_img(s_img)

# Optional dilation
kernel = np.ones((5, 5), np.uint8)
iteration_dilate = 3
fe_dil = cv2.dilate(fe_norm, kernel, iterations=iteration_dilate)
ca_dil = cv2.dilate(ca_norm, kernel, iterations=iteration_dilate)
s_dil  = cv2.dilate(s_norm, kernel, iterations=iteration_dilate)

# === Composite creator ===
def make_rgb(r, g, b):
    return np.stack([r, g, b], axis=-1)

# === Prepare 3 rows of 4 images ===
image_grid = [
    [normalize_img(fe_img), normalize_img(ca_img), normalize_img(s_img), make_rgb(normalize_img(fe_img), normalize_img(ca_img), normalize_img(s_img))],
    [fe_norm, ca_norm, s_norm, make_rgb(fe_norm, ca_norm, s_norm)],
    [fe_dil, ca_dil, s_dil, make_rgb(fe_dil, ca_dil, s_dil)],
]

titles = [
    ["Fe (Original)", "Ca (Original)", "S (Original)", "RGB (Original)"],
    ["Fe (Normalized)", "Ca (Normalized)", "S (Normalized)", "RGB (Normalized)"],
    ["Fe (Dilated)", "Ca (Dilated)", "S (Dilated)", "RGB (Dilated)"],
]

# === Display grid using Matplotlib ===
fig, axs = plt.subplots(3, 4, figsize=(16, 12))
for row in range(3):
    for col in range(4):
        ax = axs[row, col]
        img = image_grid[row][col]
        if len(img.shape) == 2:
            ax.imshow(img, cmap='gray')
        else:
            ax.imshow(img)
        ax.set_title(titles[row][col])
        ax.axis('off')
plt.tight_layout()
plt.show()

# === Blob detection ===
params = cv2.SimpleBlobDetector_Params()
params.minThreshold = 100
params.maxThreshold = 255
params.filterByArea = True
params.minArea = 200
params.maxArea = 50000
params.filterByCircularity = False
params.filterByConvexity = False
params.filterByInertia = False
params.filterByColor = True
params.blobColor = 255
detector = cv2.SimpleBlobDetector_create(params)

def find_all_blobs(img_norm, img_orig):
    keypoints = detector.detect(img_norm)
    blobs = []
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        radius = int(kp.size / 2)
        mask = np.zeros(img_orig.shape, dtype=np.uint8)
        cv2.rectangle(mask, (x - radius, y - radius), (x + radius, y + radius), 255, thickness=-1)
        vals = img_orig[mask == 255]
        if vals.size > 0:
            blob = {
                'center': (x, y),
                'radius': radius,
                'size': kp.size,
                'max_intensity': vals.max()
            }
            blobs.append(blob)
    return blobs

# === Final RGB for interactive overlay ===
merged_rgb = make_rgb(fe_norm, ca_norm, s_norm)

# === Prepare images and colors for overlay ===
images = {'Fe': (fe_dil, fe_img), 'Ca': (ca_dil, ca_img), 'S': (s_dil, s_img)}
colors = {'Fe': 'red', 'Ca': 'green', 'S': 'blue'}

fig = go.Figure()
fig.add_trace(go.Image(z=merged_rgb, name="Merged RGB Image"))

hover_traces = []

for name, (norm_img, orig_img) in images.items():
    blobs = find_all_blobs(norm_img, orig_img)
    print(f"{name} - {len(blobs)} blob(s) found.")

    for i, blob in enumerate(blobs):
        x, y = blob['center']
        r = blob['radius']
        box_x = x - r
        box_y = y - r
        box_size = 2 * r

        hover_text = (
            f"<b>{name} Blob #{i+1}</b><br>"
            f"Element: {name}<br>"
            f"Center: ({x}, {y})<br>"
            f"Top-left: ({box_x}, {box_y})<br>"
            f"Box size: {box_size} x {box_size} px<br>"
            f"Radius: {r} px<br>"
            f"Size (diameter): {blob['size']:.1f}<br>"
            f"Max intensity: {blob['max_intensity']:.3f}"
        )

        # Draw box
        box_xs = [box_x, box_x + box_size, box_x + box_size, box_x, box_x, None]
        box_ys = [box_y, box_y, box_y + box_size, box_y + box_size, box_y, None]

        hover_traces.append(go.Scatter(
            x=box_xs,
            y=box_ys,
            mode='lines',
            line=dict(color=colors[name], width=2),
            name=name,
            legendgroup=name,
            showlegend=False,
            hoverinfo='skip'
        ))

        hover_traces.append(go.Scatter(
            x=[x], y=[y],
            mode='markers',
            marker=dict(size=10, color=colors[name], opacity=0.5),
            hoverinfo='text',
            hovertext=hover_text,
            showlegend=(i == 0),
            name=name,
            legendgroup=name
        ))

for trace in hover_traces:
    fig.add_trace(trace)

fig.update_layout(
    title="Merged Fe (Red), Ca (Green), S (Blue) Image with Blob Overlays",
    xaxis=dict(scaleanchor="y", scaleratio=1, autorange=True, showgrid=False, zeroline=False, showticklabels=True, title="X (pixels)"),
    yaxis=dict(autorange='reversed', showgrid=False, zeroline=False, showticklabels=True, title="Y (pixels)"),
    hovermode="closest",
    width=1200,
    height=600,
    legend=dict(title="Element", itemsizing='constant', itemclick='toggle', itemdoubleclick='toggleothers')
)

fig.show()


In [ ]:
import os
import cv2
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib.gridspec import GridSpec

def graphing_elements(directory):
    def normalize_img(img):
        img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
        norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        return norm_img.astype(np.uint8)

    images = {}
    shapes = []

    for filename in os.listdir(directory):
        if not filename.lower().endswith((".tiff", ".tif")):
            print(f"Skipping non-tiff file: {filename}")
            continue
        if "composite" in filename.lower():
            print(f"Skipping composite file: {filename}")
            continue

        filepath = os.path.join(directory, filename)
        try:
            img = tiff.imread(filepath).astype(np.float32)
        except Exception as e:
            print(f"Skipping file '{filename}' due to read error: {e}")
            continue

        if img.ndim == 2:
            images[filename] = img
            shapes.append(img.shape)
        else:
            print(f"Skipping file '{filename}' because it is not single-channel (shape: {img.shape})")

    if not images:
        print("No valid TIFF images found.")
        return

    shape_counts = Counter(shapes)
    majority_shape = shape_counts.most_common(1)[0][0]
    print(f"Majority image shape is {majority_shape} (occurs {shape_counts[majority_shape]} times)")

    for fname, img in images.items():
        if img.shape != majority_shape:
            resized = cv2.resize(img, (majority_shape[1], majority_shape[0]), interpolation=cv2.INTER_LINEAR)
            images[fname] = resized
            print(f"Resized '{fname}' from {img.shape} to {majority_shape}")

    norm_images = {fname: normalize_img(img) for fname, img in images.items()}

    kernel = np.ones((5, 5), np.uint8)
    iteration_dilate = 3
    dilated_images = {fname: cv2.dilate(norm, kernel, iterations=iteration_dilate) for fname, norm in norm_images.items()}

    original_combined = np.zeros(majority_shape, dtype=np.float32)
    normalized_combined = np.zeros(majority_shape, dtype=np.uint8)
    dilated_combined = np.zeros(majority_shape, dtype=np.uint8)

    for img in images.values():
        original_combined = np.maximum(original_combined, img)

    for img in norm_images.values():
        normalized_combined = np.maximum(normalized_combined, img)

    for img in dilated_images.values():
        dilated_combined = np.maximum(dilated_combined, img)

    original_display = normalize_img(original_combined)

    # Prepare the images for each row: combined + all individual
    rows = [
        ("Original Combined", original_display, images),
        ("Normalized Combined", normalized_combined, norm_images),
        ("Dilated Combined", dilated_combined, dilated_images)
    ]

    n_individual = len(images)
    fig_height = 3 * 3  # 3 rows, 3 inches each roughly
    fig_width = (1 + n_individual) * 3  # combined + all individual images

    fig = plt.figure(figsize=(fig_width, fig_height))
    gs = GridSpec(3, 1 + n_individual, figure=fig, wspace=0.1, hspace=0.3)

    for row_idx, (title, combined_img, individual_dict) in enumerate(rows):
        # Combined image on the left
        ax_combined = fig.add_subplot(gs[row_idx, 0])
        ax_combined.imshow(combined_img, cmap='gray')
        ax_combined.set_title(title)
        ax_combined.axis('off')

        # Individual images to the right
        for i, (fname, img) in enumerate(individual_dict.items()):
            ax = fig.add_subplot(gs[row_idx, i+1])
            ax.imshow(img, cmap='gray')
            ax.set_title(fname, fontsize=8)
            ax.axis('off')

    plt.show()


In [ ]:
directory = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um"
graphing_elements(directory)

In [ ]:
import cv2
import numpy as np
import tifffile as tiff
import plotly.graph_objects as go

# === Load original float images ===
fe_img = tiff.imread("/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Fe_merged.tiff").astype(np.float32)
ca_img = tiff.imread("/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Ca_merged.tiff").astype(np.float32)
s_img  = tiff.imread("/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_S_merged.tiff").astype(np.float32)

# === Normalize to uint8 ===
def normalize_img(img):
    norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    return norm_img.astype(np.uint8)

fe_norm = normalize_img(fe_img)
ca_norm = normalize_img(ca_img)
s_norm  = normalize_img(s_img)

# Optional dilation
kernel = np.ones((5, 5), np.uint8)
iteration_dilate = 3
fe_dil = cv2.dilate(fe_norm, kernel, iterations=iteration_dilate)
ca_dil = cv2.dilate(ca_norm, kernel, iterations=iteration_dilate)
s_dil  = cv2.dilate(s_norm, kernel, iterations=iteration_dilate)

# === Composite creator ===
def make_rgb(r, g, b):
    return np.stack([r, g, b], axis=-1)

merged_rgb = make_rgb(fe_norm, ca_norm, s_norm)

# === Blob detection with configurable minThreshold ===
def detect_blobs(img_norm, img_orig, min_thresh):
    params = cv2.SimpleBlobDetector_Params()
    params.minThreshold = min_thresh
    params.maxThreshold = 255
    params.filterByArea = True
    params.minArea = 200
    params.maxArea = 50000
    params.filterByCircularity = False
    params.filterByConvexity = False
    params.filterByInertia = False
    params.filterByColor = True
    params.blobColor = 255
    detector = cv2.SimpleBlobDetector_create(params)

    keypoints = detector.detect(img_norm)
    blobs = []
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        radius = int(kp.size / 2)
        mask = np.zeros(img_orig.shape, dtype=np.uint8)
        cv2.rectangle(mask, (x - radius, y - radius), (x + radius, y + radius), 255, thickness=-1)
        vals = img_orig[mask == 255]
        if vals.size > 0:
            blobs.append({
                'center': (x, y),
                'radius': radius,
                'size': kp.size,
                'max_intensity': vals.max()
            })
    return blobs

# Elements to process with their dilated and original images and colors
elements = {
    'Fe': (fe_dil, fe_img, 'red'),
    'Ca': (ca_dil, ca_img, 'green'),
    'S':  (s_dil, s_img, 'blue')
}

min_thresh_vals = list(range(50, 151, 10))

fig = go.Figure()
fig.add_trace(go.Image(z=merged_rgb, name="Merged RGB Image"))

# Add dummy traces to preserve legend interactivity across all slider steps
for element_name, (_, _, color) in elements.items():
    dummy_trace = go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=10, color=color, opacity=0),
        name=element_name,
        legendgroup=element_name,
        showlegend=True,
        hoverinfo='skip',
        visible=True
    )
    fig.add_trace(dummy_trace)

# Precompute blob traces grouped by threshold
threshold_traces = {thresh: [] for thresh in min_thresh_vals}
legend_flags = {el: False for el in elements}

for thresh in min_thresh_vals:
    for element_name, (norm_img, orig_img, color) in elements.items():
        blobs = detect_blobs(norm_img, orig_img, thresh)
        print(f"{element_name} @ minThreshold={thresh} : {len(blobs)} blobs")
        for i, blob in enumerate(blobs):
            x, y = blob['center']
            r = blob['radius']
            box_x = x - r
            box_y = y - r
            box_size = 2 * r

            box_trace = go.Scatter(
                x=[box_x, box_x + box_size, box_x + box_size, box_x, box_x, None],
                y=[box_y, box_y, box_y + box_size, box_y + box_size, box_y, None],
                mode='lines',
                line=dict(color=color, width=2),
                name=element_name,
                legendgroup=element_name,
                showlegend=False,
                hoverinfo='skip',
                visible=(thresh == 100),
            )
            threshold_traces[thresh].append(box_trace)

            marker_trace = go.Scatter(
                x=[x], y=[y],
                mode='markers',
                marker=dict(size=10, color=color, opacity=0.5),
                hoverinfo='text',
                hovertext=(
                    f"<b>{element_name} Blob #{i+1}</b><br>"
                    f"Element: {element_name}<br>"
                    f"Center: ({x}, {y})<br>"
                    f"Top-left: ({box_x}, {box_y})<br>"
                    f"Box size: {box_size} x {box_size} px<br>"
                    f"Box area: {box_size * box_size} px²<br>"
                    f"Max intensity: {blob['max_intensity']:.3f}"
                ),
                name=element_name,
                legendgroup=element_name,
                showlegend=False,
                visible=(thresh == 100),
            )
            threshold_traces[thresh].append(marker_trace)

for traces in threshold_traces.values():
    for trace in traces:
        fig.add_trace(trace)

steps = []
# Index offset to skip image + dummy traces
offset = 1 + len(elements)
for i, thresh in enumerate(min_thresh_vals):
    # Start with image + all dummy traces visible
    visible = [True] * offset + [False] * (len(fig.data) - offset)
    trace_list = threshold_traces[thresh]
    for trace in trace_list:
        idx = fig.data.index(trace)
        visible[idx] = True
    steps.append(dict(
        method="update",
        args=[{"visible": visible}],
        label=str(thresh),
    ))

fig.update_layout(
    sliders=[dict(
        active=min_thresh_vals.index(100),
        currentvalue={"prefix": "minThreshold: "},
        pad={"t": 50},
        steps=steps,
    )],
    updatemenus=[dict(
        type="buttons",
        buttons=[dict(
            label="Reset to 100",
            method="update",
            args=[{"visible": [True] * offset + [t in threshold_traces[100] for t in fig.data[offset:]]}]
        )],
        pad={"t": 20},
        showactive=False,
        x=0.0,
        y=1.15,
        xanchor='left',
        yanchor='top'
    )],
    title="Merged Fe (Red), Ca (Green), S (Blue) Image with Blob Overlays and minThreshold Slider",
    xaxis=dict(scaleanchor="y", scaleratio=1, autorange=True, showgrid=False, zeroline=False, showticklabels=True, title="X (pixels)"),
    yaxis=dict(autorange='reversed', showgrid=False, zeroline=False, showticklabels=True, title="Y (pixels)"),
    hovermode="closest",
    width=1200,
    height=600,
    legend=dict(title="Element", itemsizing='constant', itemclick='toggle', itemdoubleclick='toggleothers')
)

fig.show()


In [ ]:
import os
import cv2
import numpy as np
import tifffile as tiff
import plotly.graph_objects as go

def analyze_blobs_in_directory(directory):
    def extract_element_name(filename):
        return os.path.splitext(filename)[0]

    def normalize_img(img):
        img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
        norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        return norm_img.astype(np.uint8)

    images = {}
    shapes = []
    for filename in os.listdir(directory):
        if not filename.lower().endswith((".tiff", ".tif")):
            print(f"Skipping non-tiff file: {filename}")
            continue
        if "composite" in filename.lower():
            print(f"Skipping composite file: {filename}")
            continue

        filepath = os.path.join(directory, filename)
        try:
            img = tiff.imread(filepath).astype(np.float32)
        except Exception as e:
            print(f"Skipping file '{filename}' due to read error: {e}")
            continue

        if img.ndim == 2:
            images[filename] = img
            shapes.append(img.shape)
        else:
            print(f"Skipping file '{filename}' because it is not single-channel (shape: {img.shape})")

    if not images:
        print("No valid TIFF images found.")
        return

    # Find majority shape
    from collections import Counter
    shape_counts = Counter(shapes)
    majority_shape = shape_counts.most_common(1)[0][0]
    print(f"Majority image shape is {majority_shape} (occurs {shape_counts[majority_shape]} times)")

    # Resize images to majority shape if necessary
    for fname, img in images.items():
        if img.shape != majority_shape:
            resized = cv2.resize(img, (majority_shape[1], majority_shape[0]), interpolation=cv2.INTER_LINEAR)
            images[fname] = resized
            print(f"Resized '{fname}' from {img.shape} to {majority_shape}")

    norm_images = {fname: normalize_img(img) for fname, img in images.items()}

    kernel = np.ones((5, 5), np.uint8)
    iteration_dilate = 3
    dilated_images = {fname: cv2.dilate(norm, kernel, iterations=iteration_dilate) for fname, norm in norm_images.items()}

    rgb_files = list(norm_images.keys())[:3]
    channels = [norm_images[fname] for fname in rgb_files]
    while len(channels) < 3:
        channels.append(np.zeros_like(next(iter(norm_images.values()))))
    merged_rgb = np.stack(channels, axis=-1)

    # --- rest of your function unchanged ---
    def detect_blobs(img_norm, img_orig, min_thresh):
        params = cv2.SimpleBlobDetector_Params()
        params.minThreshold = min_thresh
        params.maxThreshold = 255
        params.filterByArea = True
        params.minArea = 200
        params.maxArea = 50000
        params.filterByCircularity = False
        params.filterByConvexity = False
        params.filterByInertia = False
        params.filterByColor = True
        params.blobColor = 255
        detector = cv2.SimpleBlobDetector_create(params)

        keypoints = detector.detect(img_norm)
        blobs = []
        for kp in keypoints:
            x, y = int(kp.pt[0]), int(kp.pt[1])
            radius = int(kp.size / 2)
            mask = np.zeros(img_orig.shape, dtype=np.uint8)
            cv2.rectangle(mask, (x - radius, y - radius), (x + radius, y + radius), 255, thickness=-1)
            vals = img_orig[mask == 255]
            vals_dilated = img_norm[mask == 255]
            if vals.size > 0:
                blobs.append({
                    'center': (x, y),
                    'radius': radius,
                    'size': kp.size,
                    'max_intensity': vals.max(),
                    'mean_intensity': vals.mean(),
                    'mean_dilation': float(vals_dilated.mean())
                })
        return blobs

    min_thresh_vals = list(range(50, 151, 10))
    color_cycle = ['red', 'green', 'blue', 'orange', 'purple', 'cyan', 'magenta', 'lime']

    fig = go.Figure()
    fig.add_trace(go.Image(z=merged_rgb, name="Merged RGB Image"))

    for i, filename in enumerate(images.keys()):
        element_name = extract_element_name(filename)
        color = color_cycle[i % len(color_cycle)]
        fig.add_trace(go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=10, color=color, opacity=0.7),
            name=element_name,
            legendgroup=element_name,
            showlegend=True,
            hoverinfo='skip'
        ))

    threshold_traces = {thresh: [] for thresh in min_thresh_vals}

    for thresh in min_thresh_vals:
        for i, fname in enumerate(images.keys()):
            norm_img = dilated_images[fname]
            orig_img = images[fname]
            blobs = detect_blobs(norm_img, orig_img, thresh)
            element_name = extract_element_name(fname)
            color = color_cycle[i % len(color_cycle)]

            for j, blob in enumerate(blobs):
                x, y = blob['center']
                r = blob['radius']
                box_x = x - r
                box_y = y - r
                box_size = 2 * r

                threshold_traces[thresh].append(go.Scatter(
                    x=[box_x, box_x + box_size, box_x + box_size, box_x, box_x, None],
                    y=[box_y, box_y, box_y + box_size, box_y + box_size, box_y, None],
                    mode='lines',
                    line=dict(color=color, width=2),
                    name=element_name,
                    legendgroup=element_name,
                    showlegend=False,
                    hoverinfo='skip',
                    visible=(thresh == 100)
                ))

                threshold_traces[thresh].append(go.Scatter(
                    x=[x], y=[y],
                    mode='markers',
                    marker=dict(size=10, color=color, opacity=0.5),
                    hoverinfo='text',
                    hovertext=(
                        f"<b>{element_name} Blob #{j+1}</b><br>"
                        f"File: {fname}<br>"
                        f"Center: ({x}, {y})<br>"
                        f"Top-left: ({box_x}, {box_y})<br>"
                        f"Box size: {box_size} x {box_size} px<br>"
                        f"Box area: {box_size * box_size} px²<br>"
                        f"Max intensity: {blob['max_intensity']:.3f}<br>"
                        f"Mean intensity: {blob['mean_intensity']:.3f}<br>"
                        f"Mean dilation intensity: {blob['mean_dilation']:.1f}"
                    ),
                    name=element_name,
                    legendgroup=element_name,
                    showlegend=False,
                    visible=(thresh == 100)
                ))

    for traces in threshold_traces.values():
        for trace in traces:
            fig.add_trace(trace)

    steps = []
    offset = 1 + len(images)
    for i, thresh in enumerate(min_thresh_vals):
        visible = [True] * offset + [False] * (len(fig.data) - offset)
        for trace in threshold_traces[thresh]:
            idx = fig.data.index(trace)
            visible[idx] = True
        steps.append(dict(
            method="update",
            args=[{"visible": visible}],
            label=str(thresh),
        ))

    fig.update_layout(
        sliders=[dict(
            active=min_thresh_vals.index(100) if 100 in min_thresh_vals else 0,
            currentvalue={"prefix": "minThreshold: "},
            pad={"t": 50},
            steps=steps,
        )],
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(
                label="Reset to 100",
                method="update",
                args=[{"visible": [True] * offset + [t in threshold_traces.get(100, []) for t in fig.data[offset:]]}]
            )],
            pad={"t": 20},
            showactive=False,
            x=0.0,
            y=1.15,
            xanchor='left',
            yanchor='top'
        )],
        title="Merged Elements Image with Blob Overlays and minThreshold Slider",
        xaxis=dict(scaleanchor="y", scaleratio=1, autorange=True, showgrid=False, zeroline=False, showticklabels=True, title="X (pixels)"),
        yaxis=dict(autorange='reversed', showgrid=False, zeroline=False, showticklabels=True, title="Y (pixels)"),
        hovermode="closest",
        width=1200,
        height=600,
        legend=dict(title="Element", itemsizing='constant', itemclick='toggle', itemdoubleclick='toggleothers')
    )

    fig.show()


In [ ]:
directory = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um"

analyze_blobs_in_directory(directory)

In [ ]:
# 

In [ ]:
def three_element_threshold_graph(directory, selected_files):
    import os
    import cv2
    import numpy as np
    import tifffile as tiff
    import plotly.graph_objects as go
    from collections import Counter

    def normalize_img(img):
        img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
        norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        return norm_img.astype(np.uint8)

    def detect_blobs(img_norm, img_orig, min_thresh):
        params = cv2.SimpleBlobDetector_Params()
        params.minThreshold = min_thresh
        params.maxThreshold = 255
        params.filterByArea = True
        params.minArea = 200
        params.maxArea = 50000
        params.filterByCircularity = False
        params.filterByConvexity = False
        params.filterByInertia = False
        params.filterByColor = True
        params.blobColor = 255
        detector = cv2.SimpleBlobDetector_create(params)

        keypoints = detector.detect(img_norm)
        blobs = []
        for kp in keypoints:
            x, y = int(kp.pt[0]), int(kp.pt[1])
            radius = int(kp.size / 2)
            mask = np.zeros(img_orig.shape, dtype=np.uint8)
            cv2.rectangle(mask, (x - radius, y - radius), (x + radius, y + radius), 255, thickness=-1)
            vals = img_orig[mask == 255]
            vals_dilated = img_norm[mask == 255]
            if vals.size > 0:
                blobs.append({
                    'center': (x, y),
                    'radius': radius,
                    'size': kp.size,
                    'max_intensity': vals.max(),
                    'mean_intensity': vals.mean(),
                    'mean_dilation': float(vals_dilated.mean())
                })
        return blobs

    selected_images = {}
    shapes = []
    for fname in selected_files:
        filepath = os.path.join(directory, fname)
        img = tiff.imread(filepath).astype(np.float32)
        selected_images[fname] = img
        shapes.append(img.shape)

    majority_shape = Counter(shapes).most_common(1)[0][0]
    for fname in selected_images:
        if selected_images[fname].shape != majority_shape:
            selected_images[fname] = cv2.resize(
                selected_images[fname],
                (majority_shape[1], majority_shape[0]),
                interpolation=cv2.INTER_LINEAR
            )

    norm_images = {fname: normalize_img(img) for fname, img in selected_images.items()}
    kernel = np.ones((5, 5), np.uint8)
    dilated_images = {fname: cv2.dilate(norm, kernel, iterations=3) for fname, norm in norm_images.items()}

    channels = [norm_images[fname] for fname in selected_files]
    merged_rgb = np.stack(channels, axis=-1)

    color_cycle = ['red', 'green', 'blue']
    min_thresh_vals = list(range(50, 151, 10))

    fig = go.Figure()
    fig.add_trace(go.Image(z=merged_rgb, name="Merged RGB Image"))

    # Legend proxies
    for i, fname in enumerate(selected_files):
        element_name = os.path.splitext(fname)[0]
        color = color_cycle[i]
        fig.add_trace(go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=10, color=color, opacity=0.7),
            name=element_name,
            legendgroup=element_name,
            showlegend=True,
            hoverinfo='skip'
        ))

    # Collect blobs traces per element and threshold
    element_blob_trace_indices = []

    for i, fname in enumerate(selected_files):
        element_name = os.path.splitext(fname)[0]
        norm_img = dilated_images[fname]
        orig_img = selected_images[fname]

        thresh_trace_indices = []
        for thresh in min_thresh_vals:
            blobs = detect_blobs(norm_img, orig_img, thresh)
            this_thresh_indices = []
            color = color_cycle[i]

            for j, blob in enumerate(blobs):
                x, y = blob['center']
                r = blob['radius']
                box_x = x - r
                box_y = y - r
                box_size = 2 * r

                line_trace = go.Scatter(
                    x=[box_x, box_x + box_size, box_x + box_size, box_x, box_x, None],
                    y=[box_y, box_y, box_y + box_size, box_y + box_size, box_y, None],
                    mode='lines',
                    line=dict(color=color, width=2),
                    name=element_name,
                    legendgroup=element_name,
                    showlegend=False,
                    hoverinfo='skip',
                    visible=(thresh == 100)
                )
                fig.add_trace(line_trace)
                this_thresh_indices.append(len(fig.data) - 1)

                marker_trace = go.Scatter(
                    x=[x], y=[y],
                    mode='markers',
                    marker=dict(size=10, color=color, opacity=0.5),
                    hoverinfo='text',
                    hovertext=(
                        f"<b>{element_name} Blob #{j+1}</b><br>"
                        f"File: {fname}<br>"
                        f"Center: ({x}, {y})<br>"
                        f"Top-left: ({box_x}, {box_y})<br>"
                        f"Box size: {box_size} x {box_size} px<br>"
                        f"Box area: {box_size * box_size} px²<br>"
                        f"Max intensity: {blob['max_intensity']:.3f}<br>"
                        f"Mean intensity: {blob['mean_intensity']:.3f}<br>"
                        f"Mean dilation intensity: {blob['mean_dilation']:.1f}"
                    ),
                    name=element_name,
                    legendgroup=element_name,
                    showlegend=False,
                    visible=(thresh == 100)
                )
                fig.add_trace(marker_trace)
                this_thresh_indices.append(len(fig.data) - 1)

            thresh_trace_indices.append(this_thresh_indices)
        element_blob_trace_indices.append(thresh_trace_indices)

    # Track current thresholds per element (default 100)
    current_thresholds = [min_thresh_vals.index(100)] * len(selected_files)

    # Helper to build visibility based on current_thresholds
    def build_visibility():
        vis = [True] * (1 + len(selected_files))  # merged + legend traces
        for el_i, thresh_traces in enumerate(element_blob_trace_indices):
            selected_thresh_idx = current_thresholds[el_i]
            for th_i, trace_indices in enumerate(thresh_traces):
                show = (th_i == selected_thresh_idx)
                for idx in trace_indices:
                    while len(vis) <= idx:
                        vis.append(False)
                    vis[idx] = show
        return vis

    # Build updatemenus buttons for each element’s thresholds
    updatemenus = []
    menu_y_start = -0.15
    menu_y_spacing = 0.1
    for e_i, fname in enumerate(selected_files):
        element_name = os.path.splitext(fname)[0]
        buttons = []
        for t_i, thresh in enumerate(min_thresh_vals):
            def make_update_func(e_i_inner=e_i, t_i_inner=t_i):
                def update_visibility():
                    current_thresholds[e_i_inner] = t_i_inner
                    return [{"visible": build_visibility()}]
                return update_visibility

            buttons.append(dict(
                method="restyle",
                args=[{"visible": build_visibility()}],  # Will be updated below dynamically
                label=str(thresh),
            ))

        # We'll patch buttons' args dynamically below for correct visibilities:
        # Patch button args with correct visibility arrays for each threshold
        for idx, thresh in enumerate(min_thresh_vals):
            current_thresholds[e_i] = idx
            vis = build_visibility()
            buttons[idx]["args"] = [{"visible": vis}]

        updatemenus.append(dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.05 + e_i * 0.3,
            xanchor="left",
            y=menu_y_start + 0.15,  # Move the menu button higher to allow dropdown opening upward
            yanchor="bottom",       # Dropdown opens upwards now
            pad={"r": 10, "t": 10},
            active=current_thresholds[e_i],
            bgcolor='lightgray',
            bordercolor='gray',
            borderwidth=1,
            font=dict(size=12),
        ))

    fig.update_layout(
        updatemenus=updatemenus,
        title="Independent Blob Threshold Controls - Use Dropdowns Below",
        xaxis=dict(scaleanchor="y", scaleratio=1, autorange=True, title="X"),
        yaxis=dict(autorange='reversed', title="Y"),
        height=700,
        width=1200,
        hovermode="closest",
        legend=dict(title="Elements", itemsizing='constant'),
    )

    fig.show()


In [ ]:
three_element_threshold_graph(directory, [
    "mosaic_200_Cu_merged.tiff", "mosaic_200_Fe_merged.tiff", "mosaic_200_S_merged.tiff"
])


In [ ]:
^fix this it is still resetting the other boxes 

In [ ]:
import os

for item in os.listdir(directory):
    print(item)


In [ ]:
#Other Data Direcotry 

import os
import tifffile as tiff
import matplotlib.pyplot as plt
import math

# === Set your directory here ===
directory = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_175um_stitched"

def display_tiff_images(directory):
    if not os.path.isdir(directory):
        print("Invalid directory.")
        return

    tiff_files = [f for f in os.listdir(directory) if f.lower().endswith(('.tif', '.tiff'))]

    if not tiff_files:
        print("No TIFF images found.")
        return

    images_to_plot = []

    for filename in tiff_files:
        filepath = os.path.join(directory, filename)
        try:
            image = tiff.imread(filepath)

            if image.ndim == 2:
                images_to_plot.append((image, f"{filename} (slice 1)"))
            elif image.ndim == 3:
                for i in range(image.shape[0]):
                    images_to_plot.append((image[i], f"{filename} (slice {i + 1})"))
            else:
                print(f"Skipping {filename}: unsupported shape {image.shape}")
        except Exception as e:
            print(f"Failed to load {filename}: {e}")

    # === Plot in rows of 5 ===
    cols = 5
    rows = math.ceil(len(images_to_plot) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()

    for i, (img, title) in enumerate(images_to_plot):
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(title, fontsize=8)
        axes[i].axis('off')

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

# === Run the function ===
display_tiff_images(directory)


In [ ]:
directory = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_175um_stitched"
graphing_elements(directory)

In [ ]:
directory = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_175um_stitched"

analyze_blobs_in_directory(directory)

In [ ]:
add union code and the 3 sliders with threshold for each element. 